In [18]:
from langchain_core.tools import InjectedToolArg
from langchain_core.tools import tool
from typing import Annotated
from langchain_core.messages import HumanMessage
import requests
import sys,os
sys.path.append(os.path.abspath(".."))
from load_model import *

In [27]:
# tool create

@tool
def get_conversion_factor(base_currency:str, target_currency:str) -> float:
    """
    This function fetches the currency conversion factor between a given base currency and a target currency
    """

    url = f'https://v6.exchangerate-api.com/v6/bc8e4a213c53efffc2aab68c/pair/{base_currency}/{target_currency}'

    response = requests.get(url)


In [28]:
@tool 
def convert(base_currency_value:int, conversion_rate: Annotated[float,InjectedToolArg]) -> float:
    """
    given a currency conversion rate this function calculates the target currency value from a given base currency value
    """
    return base_currency_value * conversion_rate

In [29]:
convert.args

{'base_currency_value': {'title': 'Base Currency Value', 'type': 'integer'},
 'conversion_rate': {'title': 'Conversion Rate', 'type': 'number'}}

In [30]:
get_conversion_factor.invoke({'base_currency':'USD', 'target_currency':'INR'})

In [31]:
convert.invoke({'base_currency_value':10, 'conversion_rate':85.16})

851.5999999999999

In [32]:
llm = Tiny_llm()

In [33]:
llm_with_tools = llm.bind_tools([get_conversion_factor,convert])

NotImplementedError: 

In [19]:
message = [HumanMessage('What is the conversion factor between INR and USD, and based on that can you convert 10 inr to usd')]

In [20]:
message

[HumanMessage(content='What is the conversion factor between INR and USD, and based on that can you convert 10 inr to usd', additional_kwargs={}, response_metadata={})]

In [22]:
ai_message = llm_with_tools.invoke(message)

NameError: name 'llm_with_tools' is not defined

In [21]:
ai_message.tool_calls

NameError: name 'ai_message' is not defined

In [23]:
import json
for tool_call in ai_message.tool_calls:
    # execute the 1st tool and get the value of conversion rate
    if tool_call['name'] == 'get_conversion_factor':
        tool_message1 = get_conversion_factor.invoke(tool_call)
        # fetch this conversion rate
        conversion_rate = json.loads(tool_message1.content)['conversion_rate']
        # append this tool message to message list
        message.append(tool_message1)
    # execute the 2nd tool using the conversion rate from tool 1
    if tool_call['name'] == 'convert':
        # fetch the current arg
        tool_call['args']['conversion_rate'] = conversion_rate
        tool_message2 = convert.invoke(tool_call)
        message.append(tool_message2)


NameError: name 'ai_message' is not defined

In [ ]:
message

In [ ]:
llm_with_tools.invoke(message).content

In [24]:
from langchain.agents import initialize_agent, AgentType

#step 5: Initialize the Agent --
agent_executor = initialize_agent(
    tools=[get_conversion_factor,convert],
    llm = llm,
    agent = AgentType.STRUCTURED_CHAT_ZERO_SHOT_REACT_DESCRIPTION,  #using ReAct pattern
    verbose = True  # show internal thinking
)

C:\Users\POOJA\AppData\Local\Temp\ipykernel_22264\4041114783.py:4: LangChainDeprecationWarning: LangChain agents will continue to be supported, but it is recommended for new use cases to be built with LangGraph. LangGraph offers a more flexible and full-featured framework for building agents, including support for tool-calling, persistence of state, and human-in-the-loop workflows. For details, refer to the `LangGraph documentation <https://langchain-ai.github.io/langgraph/>`_ as well as guides for `Migrating from AgentExecutor <https://python.langchain.com/docs/how_to/migrate_agent/>`_ and LangGraph's `Pre-built ReAct agent <https://langchain-ai.github.io/langgraph/how-tos/create-react-agent/>`_.
  agent_executor = initialize_agent(


In [26]:
# ---- Step 6: Run the Agent ----
user_query = "hi how are you?"
response = agent_executor.invoke({"input":user_query})



> Entering new AgentExecutor chain...


APIConnectionError: Connection error.